<a href="https://colab.research.google.com/github/fatehOurghi/face-detection-evaluation/blob/main/notebook_FD_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of face detection algorithms

This colab notebook will be used to evaluate different face detection algorithms including:
1.   TinaFace (best of Wider Face)
2.   DSFD (best of FDDB)
3.   SRN (best of Pascal Face)
---
Datasets used in this evaluation are:
*   Wider Face
*   FDDB
*   Pascal Face

**Machine configurations**

In [ ]:
!nvidia-smi

Mon Mar 15 06:32:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare algorithms

**DSFD**

In [11]:
# install face_detection framework
!pip install face-detection

  Created wheel for face-detection: filename=face_detection-0.2.1-cp37-none-any.whl size=21404 sha256=bf09fb4ac6074ea9a5125c8fa48266bb4555601362a3eff321da0ccf578c8712
  Stored in directory: /root/.cache/pip/wheels/7d/45/54/d117d6cc260f31a22d15cda860ff46325af314cac687848ab7
Successfully built face-detection


In [12]:
# imports
import cv2
import face_detection
import time
import os
import json
from google.colab.patches import cv2_imshow

In [13]:
# prepare detection and evaluation
def dsfd_detector_eval(detector, annotations):
  detections_list = []
  for annotation_doc in annotations:
    for annotation in annotation_doc['annotations']:
      path_to_img = f"{test_path}images/{annotation['filename']}"
      im = cv2.imread(path_to_img)
      start = time.time()
      bounding_boxes = detector.detect(im)
      inference_time = time.time() - start
      detection = {
          'filename': path_to_img,
          'inference_time': inference_time,
          'faces': bounding_boxes
      }
      detections_list.append(detection)
  return detections_list

**RetinaFace**

In [ ]:
# install retinaface
!pip install retinaface-pytorch
# Retinaface example
from retinaface.pre_trained_models import get_model

def retinaface_example_test(path_to_img="/content/drive/MyDrive/Datasets/FaceDetection/WIDERFace/test/images/22--Picnic/22_Picnic_Picnic_22_106.jpg"):
  model = get_model("resnet50_2020-07-20", max_size=2048)
  model.eval()
  im = cv2.imread(path_to_img)
  start = time.time()
  detections = model.predict_jsons(im, confidence_threshold=.5, nms_threshold=0.3)
  print("time: ", time.time() - start)
  for rect in detections:
    rectangle = rect['bbox']
    cv2.rectangle(im, (int(rectangle[0]), int(rectangle[1])), (int(rectangle[2]), int(rectangle[3])), (0,0,190), 2)
  cv2_imshow(im)

In [ ]:
retinaface_example_test()

# Load dataset

In [2]:
ROOT = '/content/drive/MyDrive/Datasets/FaceDetection/'


#default: load rectangles list
def load_annotations(path, atype='rectangle'):
  filenames = []
  annotations = []
  for filename in os.listdir(path):
    if filename.__contains__(atype):
      filenames.append(filename)
  for filename in filenames:
    raw = open(path + filename)
    doc = {
        'annotations_filename': filename,
        'annotations': json.loads(raw.read())
    }
    annotations.append(doc)
    raw.close()
  return annotations

**Load WiderFace**

In [ ]:
DATASET = "WIDERFace"
##########################################################################
DATASET_PATH = f"{ROOT}{DATASET}/"
train_path = f"{DATASET_PATH}train/"
test_path = f"{DATASET_PATH}test/"


**Load FDDB**

In [ ]:
DATASET = "FDDB"
##########################################################################
DATASET_PATH = f"{ROOT}{DATASET}/"
train_path = f"{DATASET_PATH}train/"
test_path = f"{DATASET_PATH}test/"
annotations = load_annotations(test_path + "annotations/")
# img_path = test_path + annotations[0]['annotations'][0]['filename'] + '.jpg' # example img


**Load Pascal Face**

In [ ]:
DATASET = "PascalFace"
##########################################################################
DATASET_PATH = f"{ROOT}{DATASET}/"
train_path = f"{DATASET_PATH}train/"
test_path = f"{DATASET_PATH}test/"
annotations = load_annotations(f"{test_path}annotations/")

# Test Algorithms

**TinaFace**

In [ ]:
# load model
import torch
tinaface_path = "/content/drive/MyDrive/Models/FaceDetection/Tinaface_r50_fpn_widerface.pth"
model = torch.load(tinaface_path)
print(model.keys())

**DSFD**

In [ ]:
import numpy as np
import json

def evaluate_and_write_to_file():
  # prepare detector
  detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)
  detections_list = dsfd_detector_eval(detector, annotations)
  for detection in detections_list:
    detection['faces'] = [l.tolist() for l in detection['faces']]

  mean_time = np.mean([detection['inference_time'] for detection in detections_list])

  dsfd_evaluation = {
      'dataset': 'Pascal Face',
      'algorithm': 'DSFD', 
      'mean_time': mean_time,
      'detections': detections_list
  }

  eval_pascal_dsfd = open('evaluation of DSFD on Pascal Face.json', 'w')
  eval_pascal_dsfd.write(json.dumps(dsfd_evaluation))

evaluate_and_write_to_file()